In [1]:
import os
import torch
from tqdm import *
from tools import show_tensor,change_color,judge_color
from model import LeNet

from datasets import ColoredMNIST
from torch.utils.data import DataLoader


In [7]:
BATCH_SIZE = 64
EPOCH = 10
LEARNING_RATE = 1e-3
EPS = 1e-8
BETAS = [0.9, 0.999]
WEIGHT_DECAY = 1e-4

In [11]:
data_dir = '/my/datasets/path'
env = [0.1, 0.9] # image with label = 0~4 is green with p of 0.9,and image with label = 5~9 is red with p of 0.9
colormnist = ColoredMNIST(data_dir, env) # contains len(env) dataset
dataset_train = colormnist[0]
dataset_test = colormnist[1]
train_loaders = DataLoader(dataset=dataset_train, batch_size=BATCH_SIZE, shuffle=True)
test_loaders = DataLoader(dataset=dataset_test, batch_size=BATCH_SIZE, shuffle=True)


In [9]:
def test(model, loader):
    model.eval()
    num = 0
    correct = 0
    with torch.no_grad():
        for i, (img, label) in enumerate(loader):
            pre_label = torch.argmax(model(img), dim=1)
            correct += sum(pre_label==label)
            num += len(label)
    model.train()
    return correct/num
        

In [10]:
p_red = 0.5
p_green = 0.5

lamb1 = 1e-4
lamb2 = 1e-4

model = LeNet()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE, betas = BETAS, eps = EPS, weight_decay=WEIGHT_DECAY)
loss_fn = torch.nn.CrossEntropyLoss()
loss_recon = torch.nn.MSELoss()
for epoch in range(EPOCH):
    loop = tqdm(enumerate(train_loaders), total=len(train_loaders))
    for i, (img, label) in loop:
        reverse_img = change_color(img)
        pre_label = model(img)
        pre_label_2 = model(reverse_img)
        color_vector = judge_color(img)
        red_vector = (color_vector * p_red).expand(10,len(label)).T
        green_vector = ((1-color_vector) * p_green).expand(10,len(label)).T

        # print(red_vector.shape)
        do_label =  pre_label * red_vector + pre_label_2 * green_vector

        loss_cls = loss_fn(do_label, label)
        loss_recon_1 = loss_recon(do_label, pre_label)
        loss_recon_2 = loss_recon(do_label, pre_label_2)
        loss = loss_cls + lamb1 * loss_recon_1 + lamb2 * loss_recon_2
        # print(loss, loss_cls, loss_recon_1, loss_recon_2)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loop.set_description(f'Epoch [{epoch}/{EPOCH}]')
        loop.set_postfix(loss = loss.item(), loss_cls = loss_cls.item(), loss_rec = loss_recon_1.item()+loss_recon_2.item())
    test_acc = test(model, test_loaders)
    train_acc = test(model, train_loaders)
    print('train acc: {:.5f} test acc: {:.5f}'.format(train_acc, test_acc))



# judge_color(img)
# for i in range(32):
#     show_tensor(img[i])
# show_tensor(change_color(img)[0])

Epoch [0/10]: 100%|██████████| 547/547 [00:06<00:00, 87.63it/s, loss=0.135, loss_cls=0.134, loss_rec=18.2]  


train acc: 0.94863 test acc: 0.94951


Epoch [1/10]: 100%|██████████| 547/547 [00:05<00:00, 91.49it/s, loss=0.146, loss_cls=0.143, loss_rec=21.9]  


train acc: 0.96954 test acc: 0.96954


Epoch [2/10]: 100%|██████████| 547/547 [00:06<00:00, 91.13it/s, loss=0.0291, loss_cls=0.0266, loss_rec=25.1] 


...

train acc: 0.98954 test acc: 0.98320


Epoch [8/10]: 100%|██████████| 547/547 [00:05<00:00, 93.06it/s, loss=0.0308, loss_cls=0.0278, loss_rec=29.3]  


train acc: 0.99303 test acc: 0.98597


Epoch [9/10]: 100%|██████████| 547/547 [00:05<00:00, 93.74it/s, loss=0.0625, loss_cls=0.0597, loss_rec=27.7]   


train acc: 0.99406 test acc: 0.98671


In [16]:
model.eval()
with torch.no_grad():
    for epoch in range(EPOCH):
        for i, (img, label) in enumerate(test_loaders):
            show_tensor(img[i])
            pre = model(img)
            pre = torch.argmax(pre, dim=1)
            print('label:', pre[0].item())
            break

label: 8


label: 1


label: 1


label: 8


label: 4


label: 8


label: 1


label: 9


label: 5


label: 7


In [17]:
model.eval()
with torch.no_grad():
    for epoch in range(EPOCH):
        for i, (img, label) in enumerate(train_loaders):
            show_tensor(img[i])
            pre = model(img)
            pre = torch.argmax(pre, dim=1)
            print('label:', pre[0].item())
            break

label: 1


label: 4


label: 9


label: 3


label: 1


label: 2


label: 0


label: 7


label: 2


label: 3
